In [6]:
from tasks import run_md5
from tasks import run_fastqc
from invoke import run
import time
from datetime import datetime
from core.utils import Tibanna
from core import ff_utils

wf_md5 = "md5"

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
run_md5 = raw_input("Do you wanna run md5 if missing? (md5/none)")


ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
tibanna.ff_keys['default']['server'] = 'https://data.4dnucleome.org'
raw_bucket = tibanna.s3.raw_file_bucket
out_bucket = tibanna.s3.outfile_bucket

file_url = '/search/?status=uploading&status=upload%20failed&type=FileMicroscopy'
all_files = ff_utils.get_metadata(file_url, connection=ff)['@graph']

print len(all_files)


printn = 0
counter = 0
for a_file in all_files:  
    counter += 1
    # check for deleted or weird cases
    try:
        if a_file['status'] == 'deleted':
            #print "Deleted File", a_file
            continue
    except:
        print a_file
        break
        
    if counter-printn > 100:
        print counter
        printn = counter

    file_resp = ff_utils.get_metadata(a_file['uuid'], connection=ff, frame='embedded')
    
    
    if 'FileProcessed' in file_resp['@type']:
            my_bucket = out_bucket
    else:  # covers cases of FileFastq, FileReference, FileMicroscopy
            my_bucket = raw_bucket
            
            
    # check if file is in s3
    head_info = tibanna.s3.does_key_exist(file_resp['upload_key'], my_bucket)
    if not head_info:
        print file_resp['accession'], "does not have a file in S3"
        continue
    md5 = False
    
    file_id = file_resp['accession']
    status = file_resp.get('status')
    workflows = file_resp.get('workflow_run_inputs')
    
    if file_resp.get('md5sum'):
        md5 = True
        
    # is there md5sum for gzip
    if not file_resp.get('md5sum'):
        print file_id,"does not have the md5sum calculated during upload"
        
    # Check workflows f
    md5_status = 'did_not_run'
    
    # Assumes workflow_runs come in time ordered list, and grabs the last ones for each wf run
    if workflows:
        for a_wfr in workflows:
            wfr_resp = ff_utils.get_metadata(a_wfr['uuid'], connection=ff)
            wfr_name = wfr_resp['display_title']
            if wfr_name.startswith(wf_md5):
                md5_status = wfr_resp.get('run_status') 

#     print file_resp['accession']    
#     print file_resp['lab']['@id']
#     print "status", file_resp['status']
#     print "md5_status", md5_status
#     print "md5 okay?", md5
#     print file_resp['aliases']

    if status in ["uploading", "upload failed"] and md5_status != 'complete':
        # if not, shall we run it?
        if run_md5 == 'md5':
            print 'md5 running for', file_id
            code_md5= "invoke run_md5 " + env + " " + file_resp['display_title'] + " " + file_resp['uuid']
            run(code_md5)
            print ''
            time.sleep(3)
        # user does not want it to be run, so just report
        else:
            print 'md5 run missing for', file_id
    
    elif status in ["uploading", "upload failed"] and md5_status == 'complete':
        print "ERROR:",file_id ,"There is a successful md5 run but status not switched"
    print



Do you wanna run md5 if missing? (md5/none)md5
811
object 157af41c-ff97-4df6-8d46-bb0cfc7c5b2c/4DNFII5APXQO.r3d not found on bucket elasticbeanstalk-fourfront-webprod-files
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFII5APXQO does not have a file in S3
object f0d5c051-f5e5-4758-a123-9f24e23322ca/4DNFI2NAG7CH.r3d not found on bucket elasticbeanstalk-fourfront-webprod-files
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFI2NAG7CH does not have a file in S3
4DNFI4KZIPV7 does not have the md5sum calculated during upload
md5 running for 4DNFI4KZIPV7


Traceback (most recent call last):
  File "/Users/koray/.virtualenvs/tibanna/bin/invoke", line 11, in <module>
    load_entry_point('invoke==0.17.0', 'console_scripts', 'invoke')()
  File "/Users/koray/.virtualenvs/tibanna/lib/python2.7/site-packages/invoke/program.py", line 285, in run
    self.execute()
  File "/Users/koray/.virtualenvs/tibanna/lib/python2.7/site-packages/invoke/program.py", line 400, in execute
    executor.execute(*self.tasks)
  File "/Users/koray/.virtualenvs/tibanna/lib/python2.7/site-packages/invoke/executor.py", line 132, in execute
    result = call.task(*args, **call.kwargs)
  File "/Users/koray/.virtualenvs/tibanna/lib/python2.7/site-packages/invoke/tasks.py", line 111, in __call__
    result = self.body(*args, **kwargs)
  File "/Users/koray/Github/tibanna/tasks.py", line 428, in run_md5
    file_name = meta_data['upload_key'].split('/')[-1]
KeyError: 'upload_key'


UnexpectedExit: Encountered a bad command exit code!

Command: u'invoke run_md5 fourfront-webprod 4DNFI4KZIPV7.r3d 21efd180-e86d-435e-874a-28e35f3e4870'

Exit code: 1

Stdout: already printed

Stderr: already printed

